# Data Optimization
- Entropy coding: 등장 빈도가 높은 데이터일수록 작은 벡터를, 등장 빈도가 낮은 데이터일수록 큰 벡터를 할당하여 데이터 분포를 최적화합니다.
- input: 시퀀스로 변환된 데이터, meta 데이터
- output
    - dic: 고유 ID - 콘텐츠 이름 간 매칭
    - dic2: 최적화된 벡터 - 콘텐츠 이름 간 매칭
    - match: 고유 ID - 최적화된 벡터 간 매칭
    - input_final: 최적화된 벡터로 치환된 시퀀스 데이터

In [1]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
import numpy as np

# Input data 호출
df = pd.read_csv("/data/log-data-2024/2.sequence_generate_ksc/data/input_241127.txt", sep = "\t", header = None)
df[1] = [i.split()[1:] for i in df[0]]

# 데이터 등장 빈도 도출
lit = []
for i in df[1]:
    for w in i:
        lit.append(w)
lit = dict(Counter(lit))
lit = pd.DataFrame({"content" : list(lit.keys()), "number" : list(lit.values())})
lit = lit.sort_values(by = "number", ascending = False)
lit = list(lit["content"])

# 등장 빈도가 높을수록 작은 벡터 할당
match = {}
for a, i in enumerate(lit):
    match[i] = str(a + 1)

match

{'708880.0': '1',
 '709176.0': '2',
 '645850.0': '3',
 '677366.0': '4',
 '633507.0': '5',
 '738402.0': '6',
 '274033.0': '7',
 '596461.0': '8',
 '602408.0': '9',
 '620084.0': '10',
 '841536.0': '11',
 '611908.0': '12',
 '644206.0': '13',
 '646057.0': '14',
 '598367.0': '15',
 '685192.0': '16',
 '580375.0': '17',
 '562629.0': '18',
 '738468.0': '19',
 '638767.0': '20',
 '738381.0': '21',
 '595073.0': '22',
 '629898.0': '23',
 '63079.0': '24',
 '708881.0': '25',
 '625699.0': '26',
 '593152.0': '27',
 '595649.0': '28',
 '621998.0': '29',
 '648513.0': '30',
 '301949.0': '31',
 '592963.0': '32',
 '628291.0': '33',
 '649745.0': '34',
 '631083.0': '35',
 '600220.0': '36',
 '594966.0': '37',
 '647726.0': '38',
 '683180.0': '39',
 '201280.0': '40',
 '626393.0': '41',
 '615989.0': '42',
 '682924.0': '43',
 '284639.0': '44',
 '627665.0': '45',
 '630065.0': '46',
 '211233.0': '47',
 '637330.0': '48',
 '831750.0': '49',
 '142401.0': '50',
 '294938.0': '51',
 '640339.0': '52',
 '594329.0': '53',
 '6

In [2]:
# 메타 데이터에 병합
dic = pd.read_csv("/data/log-data-2024/1.meta_rot_preprocessing_ksc/data/Meta_241127.csv")[["new_content_id", "title_name"]]
dic = dic.dropna().reset_index(drop = True)
dic["new_content_id"] = dic["new_content_id"].apply(lambda x : int(float(x)))
dic = dict(zip(dic["new_content_id"], dic["title_name"]))

opt = []
for i in tqdm(df.index):
    opt.append([match[w] for w in df[1].loc[i]])
df[2] = opt

title = []
for i in tqdm(df.index):
    title.append([dic[int(float(w))] for w in df[1].loc[i]])
df[3] = title

print("시퀀스 평균 길이: ", np.std([len(i) for i in df[2]]))
df

/home/hanyang-user/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████████████████████████████| 73505/73505 [00:01<00:00, 45520.47it/s]

시퀀스 평균 길이:  6.949006613317722


,0,1,2,3
0,0 588756.0 151926.0 588756.0,"[588756.0, 151926.0, 588756.0]","[3433, 25529, 3433]","[Madeline, Legends of Oz: Dorothy's Return, Ma..."
1,1 596461.0 596461.0 596461.0 629694.0 627713.0,"[596461.0, 596461.0, 596461.0, 629694.0, 62771...","[8, 8, 8, 131, 181]","[Twisters, Twisters, Twisters, Hotel Transylva..."
2,2 595073.0 595073.0 595071.0 612889.0,"[595073.0, 595073.0, 595071.0, 612889.0]","[22, 22, 221, 293]","[Coraline, Coraline, Spider-Man: Into the Spid..."
3,3 567397.0 184872.0 598367.0 564650.0 564650.0...,"[567397.0, 184872.0, 598367.0, 564650.0, 56465...","[87, 2955, 15, 224, 224, 75, 1525, 1525, 1525,...","[No Time to Die, The Burial, Twisters, Guardia..."
4,4 285772.0 627528.0 63079.0,"[285772.0, 627528.0, 63079.0]","[94, 55, 24]","[Illumination Presents Sing 2-Movie Pack, Sing..."
...,...,...,...,...
73500,73500 626870.0 626870.0 633123.0 601166.0 6271...,"[626870.0, 626870.0, 633123.0, 601166.0, 62719...","[10167, 10167, 1875, 1791, 1171, 30090, 3752, ...","[Pil's Adventures, Pil's Adventures, Alvin and..."
73501,73501 82732.0 633240.0 683060.0,"[82732.0, 633240.0, 683060.0]","[79, 376, 539]","[Coraline, Monster House, Anna And The Apocaly..."
73502,73502 588390.0 82633.0 622328.0 618101.0 61810...,"[588390.0, 82633.0, 622328.0, 618101.0, 618101...","[1273, 1386, 996, 6125, 6125, 372]","[There's Something About Mary, Halloween, Nope..."
73503,73503 635647.0 635647.0 635647.0,"[635647.0, 635647.0, 635647.0]","[56, 56, 56]","[Practical Magic, Practical Magic, Practical M..."


In [3]:
dic2 = {}
for i in tqdm(match):
    dic2[match[i]] = dic[int(float(i))]
dic2

100%|█████████████████████████████████| 30090/30090 [00:00<00:00, 328527.01it/s]


{'1': "It's the Great Pumpkin, Charlie Brown",
 '2': 'College Football',
 '3': 'Beetlejuice',
 '4': "Dr. Seuss' The Grinch",
 '5': 'Despicable Me 4: Includes 2 Mini Movies',
 '6': 'CNN',
 '7': 'Score!',
 '8': 'Twisters',
 '9': 'Beetlejuice Beetlejuice',
 '10': 'How the Grinch Stole Christmas',
 '11': "It's the Great Pumpkin, Charlie Brown",
 '12': 'Hotel Transylvania',
 '13': 'Despicable Me 4 (Includes 2 Mini-Movies)',
 '14': 'Elf',
 '15': 'Twisters',
 '16': 'Twilight',
 '17': 'Inside Out 2',
 '18': 'Coraline',
 '19': 'Newsmax',
 '20': 'Inside Out 2: And Digital Bonus',
 '21': 'ABC News',
 '22': 'Coraline',
 '23': 'The Wild Robot',
 '24': 'Sing 2',
 '25': 'A Charlie Brown Thanksgiving',
 '26': 'Halloween',
 '27': 'Casper',
 '28': 'Coraline',
 '29': 'Terrifier 2',
 '30': 'Beetlejuice',
 '31': 'Harry Potter 8-Film Collection',
 '32': 'Shrek',
 '33': 'The Super Mario Bros. Movie',
 '34': 'Inside Out 2 - Bonus X-Ray Edition',
 '35': 'The Fall Guy',
 '36': 'Venom: Let There Be Carnage',
 '3

In [6]:
# # 데이터 저장
# import pickle

# 생성했던 dictionary들 저장
# with open(file='/data/log-data-2024/Final_20241127/match_dict_final.pickle', mode='wb') as f:
#     pickle.dump(dic, f)
# with open(file='/data/log-data-2024/Final_20241127/match_dict_final2.pickle', mode='wb') as f:
#     pickle.dump(dic2, f)
# with open(file='/data/log-data-2024/Final_20241127/match.pickle', mode='wb') as f:
#     pickle.dump(match, f)

# # dictionary를 적용한 인풋 데이터 저장
# cf = []
# for a, i in enumerate(df[2]):
#     cf.append(str(a) + " " + " ".join(i))

# with open("/data/log-data-2024/Final_20241127/input_search_final_20241127.txt", 'w') as file:
#     for line in cf:
#         file.write(line + '\n') # 백업용 저장
        
# with open("/data/log-data-2024/5. SASRec/BSARec/src/data//input_search_final_20241127.txt", 'w') as file:
#     for line in cf:
#         file.write(line + '\n') # SASRec을 위한 저장

# Data Augmentation
- 시퀀스를 subsequence로 증강하는 데이터 전처리 코드

In [18]:
augmented = []
for i in tqdm(df[2]):
    if len(i) == 3:
        augmented.append(i)
    else:
        for w in range(3, len(i) + 1):
            augmented.append(i[:w])
augmented

100%|█████████████████████████████████| 73505/73505 [00:00<00:00, 147631.67it/s]


[['3433', '25529', '3433'],
 ['8', '8', '8'],
 ['8', '8', '8', '131'],
 ['8', '8', '8', '131', '181'],
 ['22', '22', '221'],
 ['22', '22', '221', '293'],
 ['87', '2955', '15'],
 ['87', '2955', '15', '224'],
 ['87', '2955', '15', '224', '224'],
 ['87', '2955', '15', '224', '224', '75'],
 ['87', '2955', '15', '224', '224', '75', '1525'],
 ['87', '2955', '15', '224', '224', '75', '1525', '1525'],
 ['87', '2955', '15', '224', '224', '75', '1525', '1525', '1525'],
 ['87', '2955', '15', '224', '224', '75', '1525', '1525', '1525', '16938'],
 ['87',
  '2955',
  '15',
  '224',
  '224',
  '75',
  '1525',
  '1525',
  '1525',
  '16938',
  '926'],
 ['87',
  '2955',
  '15',
  '224',
  '224',
  '75',
  '1525',
  '1525',
  '1525',
  '16938',
  '926',
  '749'],
 ['87',
  '2955',
  '15',
  '224',
  '224',
  '75',
  '1525',
  '1525',
  '1525',
  '16938',
  '926',
  '749',
  '1318'],
 ['94', '55', '24'],
 ['314', '16910', '16910'],
 ['314', '16910', '16910', '582'],
 ['314', '16910', '16910', '582', '896'

In [20]:
# # 데이터 저장

# cf = []
# for a, i in enumerate(augmented):
#     cf.append(str(a) + " " + " ".join(i))

# with open("/data/log-data-2024/Final_20241127/input_search_augmented_final_20241127.txt", 'w') as file:
#     for line in cf:
#         file.write(line + '\n') # 백업용 저장
        
# with open("/data/log-data-2024/5. SASRec/BSARec/src/data/input_search_augmented_final_20241127.txt", 'w') as file:
#     for line in cf:
#         file.write(line + '\n') # SASRec을 위한 저장